# Setup and Explore Databases

## Overview

This notebook is the first step in the lab, designed to finalize the setup of the database environment and to familiarize you with the database schemas and data. The core infrastructure, including an AlloyDB cluster and a Spanner instance, is already provisioned via Terraform. The notebook will guide you through the final setup steps for these databases and allow you to explore their structure and content. 

The key milestones you will achieve are:

- Connecting to your Google Cloud project and the provisioned databases.
- Finalizing the setup of the AlloyDB and Spanner databases.
- Importing and validating data in both AlloyDB and Spanner.
- Updating the dates in the datasets to demonstrate real-time capabilities.
- Exploring the schemas and data in both databases through various queries.

### Terraform Resources

The following pre-requisite resources were created for you by Terraform. See the [main.tf](../terraform/main.tf) file for more details on the environment configuration:

- Spanner, AlloyDB, and other related APIs
- Network components for secure communication (Custom VPC, Cloud Router, Cloud NAT Gateway, Firewall Rules, Private IP Range for Service Networking)
- AlloyDB Cluster and Instance
- Spanner Instance, Database, Schema, and Data
- Vertex AI Workbench Instance

### Google Cloud Services Used in this Notebook

This notebook utilizes the following Google Cloud services:
- Vertex AI Workbench: A Jupyter notebook-based development environment for the entire data science workflow.
- AlloyDB: A fully-managed, 100% PostgreSQL-compatible database service to power your most demanding enterprise workloads with superior performance, availability, and scale, and supercharge them with AI.
- Spanner: A fully-managed, geographically distributed relational database with unlimited scale, strong consistency, and up to 99.999% availability, bringing together relational, graph, key value, vector, and search data into a single database.
- Cloud Storage: A managed service for storing any amount of data unstructured data retrieving it as often as you like.
- IAM (Identity and Access Management): Fine-grained access control and visibility for centrally managing cloud resources.

### Logical Flow
This notebook provides step-by-step instructions to guide you through the database setup and exploration process. The logical flow is as follows:
- Basic Setup: This section includes defining notebook variables, connecting to your Google Cloud project, configuring logging, and installing dependencies.
- Helper Functions: Helper functions are defined for interacting with the REST API, AlloyDB, and Spanner to simplify the code in the subsequent sections.
- Finalize AlloyDB Setup: This section connects to the AlloyDB cluster, creates the finance database, and imports data from a SQL file in a GCS bucket. It also includes a step to validate the row counts of the imported tables.
- Update Dates: The dates in both the AlloyDB and Spanner tables are updated to reflect more recent data, which is crucial for demonstrating the real-time capabilities of the solution.
- Explore AlloyDB Data: This section provides a review of the AlloyDB schema and allows you to run queries to explore the data in the various tables.
- Explore Spanner Data: Similar to the AlloyDB section, this part of the notebook reviews the Spanner schema (including the property graph) and allows you to run both relational and graph queries to explore the data.

Let's get started!

## Basic Setup

### Review Organization Policies

If you have [Organization Policies](https://console.cloud.google.com/iam-admin/orgpolicies/list) that explicitly deny certain configurations, you will need to work with your platform team to add an exception. Specifically, you may need to override the following policies with an "Allow All" rule (see [the docs](https://cloud.google.com/resource-manager/docs/organization-policy/creating-managing-policies) for more details): `constraints/run.allowedIngress`, `constraints/iam.allowedPolicyMemberDomains`, `constraints/compute.vmExternalIpAccess`.

### Define Notebook Variables

Update the `project_id` and `region` variables below to match your environment. You can find these values in your Terraform output. You can use defaults for the rest of the project variables .

You will be prompted for two passwords:
1. The AlloyDB admin password for the `postgres` user. This is the password you defined when provisioning the Terraform environment. 
2. A new password for a least-privilege `toolbox_user` account that we will create later in the notebook. Take note of this password for later labs.

In [ ]:
# Project variables
project_id = "my-project"
region = "my-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

# AlloyDB variables
alloydb_cluster = "my-alloydb-cluster"
alloydb_instance = "my-alloydb-instance"
alloydb_database = "finance"
alloydb_password = input("Please enter the password for the AlloyDB 'postgres' database user: ")
alloydb_toolbox_user_password = input("Please choose a password for the new AlloyDB 'toolbox_user' database user: ")

# Spanner variables
spanner_instance = "my-spanner-instance"
spanner_database = "finance-graph"
session = None


In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet google-cloud-storage==2.19.0 \
                      asyncpg==0.30.0 \
                      google.cloud.alloydb.connector==1.9.0 

### Define Helper Functions

#### REST API Helper Function

In [ ]:
import requests
import google.auth
import json

# Get an access token based upon the current user
creds, _ = google.auth.default()
authed_session = google.auth.transport.requests.AuthorizedSession(creds)
access_token=creds.token

if project_id:
  authed_session.headers.update({"x-goog-user-project": project_id}) # Required to workaround a project quota bug

def rest_api_helper(
    url: str,
    http_verb: str,
    request_body: dict = None,
    params: dict = None,
    session: requests.Session = authed_session,
  ) -> dict:
  """Calls a REST API using a pre-authenticated requests Session."""

  headers = {"Content-Type": "application/json"}

  try:

    if http_verb == "GET":
      response = session.get(url, headers=headers, params=params)
    elif http_verb == "POST":
      response = session.post(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PUT":
      response = session.put(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PATCH":
      response = session.patch(url, json=request_body, headers=headers, params=params)
    elif http_verb == "DELETE":
      response = session.delete(url, headers=headers, params=params)
    else:
      raise ValueError(f"Unknown HTTP verb: {http_verb}")

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # Check if response has content before trying to parse JSON
    if response.content:
        return response.json()
    else:
        return {} # Return empty dict for empty responses (like 204 No Content)

  except requests.exceptions.RequestException as e:
      # Catch potential requests library errors (network, timeout, etc.)
      # Log detailed error information
      print(f"Request failed: {e}")
      if e.response is not None:
          print(f"Request URL: {e.request.url}")
          print(f"Request Headers: {e.request.headers}")
          print(f"Request Body: {e.request.body}")
          print(f"Response Status: {e.response.status_code}")
          print(f"Response Text: {e.response.text}")
          # Re-raise a more specific error or a custom one
          raise RuntimeError(f"API call failed with status {e.response.status_code}: {e.response.text}") from e
      else:
          raise RuntimeError(f"API call failed: {e}") from e
  except json.JSONDecodeError as e:
      print(f"Failed to decode JSON response: {e}")
      print(f"Response Text: {response.text}")
      raise RuntimeError(f"Invalid JSON received from API: {response.text}") from e


#### AlloyDB Helper Function

In [ ]:
# Create AlloyDB Query Helper Function
import sqlalchemy
from sqlalchemy import text, exc
import pandas as pd

async def run_alloydb_query(pool, sql: str, params = None, output_as_df: bool = True):
    """Executes a SQL query or statement against the database pool.

    Handles various SQL statements:
    - SELECT/WITH: Returns results as a DataFrame (if output_as_df=True)
      or ResultProxy. Supports parameters. Does not commit.
    - EXPLAIN/EXPLAIN ANALYZE: Executes the explain, returns the query plan
      as a formatted multi-line string. Ignores output_as_df.
      Supports parameters. Does not commit.
    - INSERT/UPDATE/DELETE/CREATE/ALTER etc.: Executes the statement,
      commits the transaction, logs info, and returns the ResultProxy.
      Supports single or bulk parameters (executemany).

    Args:
      pool: An asynchronous SQLAlchemy connection pool.
      sql: A string containing the SQL query or statement template.
      params: Optional.
        - None: Execute raw SQL (Use with caution for non-SELECT/EXPLAIN).
        - dict or tuple: Parameters for a single execution.
        - list of dicts/tuples: Parameters for bulk execution (executemany).
      output_as_df (bool): If True and query is SELECT/WITH, return pandas DataFrame.
                           Ignored for EXPLAIN and non-data-returning statements.

    Returns:
      pandas.DataFrame | str | sqlalchemy.engine.Result | None:
        - DataFrame: For SELECT/WITH if output_as_df=True.
        - str: For EXPLAIN/EXPLAIN ANALYZE, containing the formatted query plan.
        - ResultProxy: For non-SELECT/WITH/EXPLAIN statements, or SELECT/WITH
                       if output_as_df=False.
        - None: If a SQLAlchemy ProgrammingError or other specific error occurs.

    Raises:
        Exception: Catches and logs `sqlalchemy.exc.ProgrammingError`, returning None.
                   May re-raise other database exceptions.

    Example Execution:
      Single SELECT:
        sql_select = "SELECT ticker, company_name from investments LIMIT 5"
        df_result = await run_alloydb_query(pool, sql_select)

      Single non-SELECT - Parameterized (Safe!):
        Parameterized INSERT:
          sql_insert = "INSERT INTO investments (ticker, company_name) VALUES (:ticker, :name)"
          params_insert = {"ticker": "NEW", "name": "New Company"}
          insert_result = await run_alloydb_query(pool, sql_insert, params_insert)

        Parameterized UPDATE:
          sql_update = "UPDATE products SET price = :price WHERE id = :product_id"
          params_update = {"price": 99.99, "product_id": 123}
          update_result = await run_alloydb_query(pool, sql_update, params_update)

      Bulk Update:
        docs = pd.DataFrame([
            {'id': 101, 'sparse_embedding': '[0.1, 0.2]'},
            {'id': 102, 'sparse_embedding': '[0.3, 0.4]'},
            # ... more rows
        ])

        update_sql_template = '''
            UPDATE products
            SET sparse_embedding = :embedding,
                sparse_embedding_model = 'BM25'
            WHERE id = :product_id
        ''' # Using named parameters :param_name

        # Prepare list of dictionaries for params
        data_to_update = [
            {"embedding": row.sparse_embedding, "product_id": row.id}
            for row in docs.itertuples(index=False)
        ]

        if data_to_update:
          bulk_result = await run_alloydb_query(pool, update_sql_template, data_to_update)
          # bulk_result is the SQLAlchemy ResultProxy

    """
    sql_lower_stripped = sql.strip().lower()
    is_select_with = sql_lower_stripped.startswith(('select', 'with'))
    is_explain = sql_lower_stripped.startswith('explain')

    # Determine if the statement is expected to return data rows or a plan
    is_data_returning = is_select_with or is_explain

    # Determine actual DataFrame output eligibility (only for SELECT/WITH)
    effective_output_as_df = output_as_df and is_select_with

    # Check if params suggest a bulk operation (for logging purposes)
    is_bulk_operation = isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], (dict, tuple, list))

    async with pool.connect() as conn:
        try:
          # Execute with or without params
          if params:
              result = await conn.execute(text(sql), params)
          else:
              # Add warning for raw SQL only if it's NOT data-returning
              #if not is_data_returning:
                  #logging.warning("Executing non-SELECT/EXPLAIN raw SQL without parameters. Ensure SQL is safe.")
              result = await conn.execute(text(sql))

          # --- Handle statements that return data or plan ---
          if is_data_returning:
              if is_explain:
                  # Fetch and format EXPLAIN output as a string
                    try:
                        plan_rows = result.fetchall()
                        # EXPLAIN output is usually text in the first column
                        query_plan = "\n".join([str(row[0]) for row in plan_rows])
                        #logging.info(f"EXPLAIN executed successfully for: {sql[:100]}...")
                        return query_plan
                    except Exception as e:
                        logging.error(f"Error fetching/formatting EXPLAIN result: {e}")
                        return None
              else: # Handle SELECT / WITH
                  if effective_output_as_df:
                      try:
                          rows = result.fetchall()
                          column_names = result.keys()
                          df = pd.DataFrame(rows, columns=column_names)
                          #logging.info(f"SELECT/WITH executed successfully, returning DataFrame for: {sql[:100]}...")
                          return df
                      except Exception as e:
                          logging.error(f"Error converting SELECT result to DataFrame: {e}")
                          logging.info(f"Returning raw ResultProxy for SELECT/WITH due to DataFrame conversion error for: {sql[:100]}...")
                          return result # Fallback to raw result
                  else:
                      # Return raw result proxy for SELECT/WITH if df output not requested
                      #logging.info(f"SELECT/WITH executed successfully, returning ResultProxy for: {sql[:100]}...")
                      return result

          # --- Handle Non-Data Returning Statements (INSERT, UPDATE, DELETE, CREATE, etc.) ---
          else:
              await conn.commit() # Commit changes ONLY for these statements
              operation_type = sql.strip().split()[0].upper()
              row_count = result.rowcount # Note: rowcount behavior varies

              if is_bulk_operation:
                  print(f"Bulk {operation_type} executed for {len(params)} items. Result rowcount: {row_count}")
              elif operation_type in ['INSERT', 'UPDATE', 'DELETE']:
                  print(f"{operation_type} statement executed successfully. {row_count} row(s) affected.")
              else: # CREATE, ALTER, etc.
                  print(f"{operation_type} statement executed successfully. Result rowcount: {row_count}")
              return result # Return the result proxy

        except exc.ProgrammingError as e:
            # Log the error with context
            logging.error(f"SQL Programming Error executing query:\nSQL: {sql[:500]}...\nParams (sample): {str(params)[:500]}...\nError: {e}")
            # Rollback might happen automatically on context exit with error, but explicit can be clearer
            # await conn.rollback() # Consider if needed based on pool/transaction settings
            return None # Return None on handled programming errors
        except Exception as e:
            # Log other unexpected errors
            logging.error(f"An unexpected error occurred during query execution:\nSQL: {sql[:500]}...\nError: {e}")
            # await conn.rollback() # Consider if needed
            raise # Re-raise unexpected errors



#### Spanner Helper Functions

In [ ]:
import pandas as pd
import time


def get_spanner_sessions(project_id = project_id, spanner_instance = spanner_instance, spanner_database = spanner_database):

  # Get session
  uri = f"https://spanner.googleapis.com/v1/projects/{project_id}/instances/{spanner_instance}/databases/{spanner_database}/sessions"
  response = rest_api_helper(uri, "GET")
  return response


# https://cloud.google.com/spanner/docs/reference/rest/v1/projects.instances.databases.sessions/create
def create_spanner_session(project_id = project_id, spanner_instance = spanner_instance, spanner_database = spanner_database):
  print("No Spanner session found. Creating a new session.")

  # Create a new session
  uri = f"https://spanner.googleapis.com/v1/projects/{project_id}/instances/{spanner_instance}/databases/{spanner_database}/sessions"
  params = {
      "database": f"projects/{project_id}/instances/{spanner_instance}/databases/{spanner_database}"
  }
  response = rest_api_helper(uri, "POST", {}, params)
  return response['name']
     

# https://cloud.google.com/spanner/docs/reference/rest/v1/projects.instances.databases.sessions/delete
def close_spanner_session(session, project_id = project_id, spanner_instance = spanner_instance, spanner_database = spanner_database):
  """
  Example:
    response = get_spanner_sessions()
    for session in response['sessions']:
      close_spanner_session(session['name'])
  """

  uri = f"https://spanner.googleapis.com/v1/{session}"
  response = rest_api_helper(uri, "DELETE", {}, {"name": f"{session}"})
  return response
     

def run_spanner_query(sql, spanner_database = spanner_database, query_options=None, create_new_session=False):
  """
  Runs a Spanner query and returns the result.

  Args:
      sql: The SQL query to execute.
      query_options: (Optional) A dictionary of advanced query options.
                    See https://cloud.google.com/spanner/docs/reference/rest/v1/projects.instances.databases.sessions/executeSql#queryoptions
                    for available options.

  Returns:
      A dictionary containing the query results.

  Ref:
      https://cloud.google.com/spanner/docs/reference/rest/v1/projects.instances.databases.sessions/executeSql
  """
  # Ensure a session exists
  # Create session
  global session
  if not session or create_new_session == True:
    session = create_spanner_session()

  # Initialize response vars
  commit_response = ""
  response = ""

  # Construct the request URL
  uri = f"https://spanner.googleapis.com/v1/{session}:executeSql"

  # Set transaction type (readOnly/readWrite) and transaction object with commit type (begin/singleUse)
  transaction_type = "readWrite" if any(x in sql.lower() for x in ["insert", "update", "delete"]) else "readOnly"
  transaction = {"begin": {"readWrite": {}}} if transaction_type == "readWrite" else {"singleUse": {"readOnly": {}}}

  request_body = {
      "sql": sql,
      "transaction": transaction
  }
  params = {
      "session": session
  }

  if query_options:
      request_body["queryOptions"] = query_options

  try:
    # Make the request
    response = rest_api_helper(uri, "POST", request_body=request_body, params = params)

  except RuntimeError as e:
    if "Session not found" in str(e):
      print(f"Session not found. Creating a new session and retrying the query...")
      return run_spanner_query(sql, spanner_database, query_options, create_new_session=True)  # Retry with a new session
    else:
      raise  # Re-raise the exception if it's not a "Session not found" error

  # Commit transaction if read/write
  if transaction_type == "readWrite":
      uri = f"https://spanner.googleapis.com/v1/{session}:commit"
      params = {
          "session": session
      }
      commit_response = rest_api_helper(uri, "POST", {"transactionId": response['metadata']['transaction']['id']}, params)
      print(f"commit_response: {commit_response}")

  # Return a DataFrame if
  if (sql.lower().strip().startswith(("select", "with", "graph"))):
    columns = [field.get('name', 'unnamed_column') for field in response['metadata']['rowType']['fields']]

    # Create DataFrame from rows
    if 'rows' in response:
      df = pd.DataFrame(response['rows'], columns=columns)
      return df
    else:
      return response

  else:
    # Return the query results
    return response


def run_spanner_ddl(ddl_array, project_id = project_id, spanner_instance = spanner_instance, spanner_database = spanner_database):
  # Create tables in Spanner
  # https://cloud.google.com/spanner/docs/reference/rest/v1/projects.instances.databases.tables/create#try-it

  uri = f"https://spanner.googleapis.com/v1/projects/{project_id}/instances/{spanner_instance}/databases/{spanner_database}/ddl"
  http_verb = "PATCH"
  request_body = {
      "statements": ddl_array
  }

  response = rest_api_helper(uri, http_verb, request_body)

  operation_name = response['name']
  uri = f"https://spanner.googleapis.com/v1/{operation_name}"

  while True:
    response = rest_api_helper(uri, "GET", {})
    if response.get("done", False):
      if response.get("error"):
        print(response.get("error"))
      else:
        print("Operation completed successfully.")
      break
    else:
      print("Operation not completed yet.")
      time.sleep(2)


## Finalize AlloyDB Setup

### Connect to the AlloyDB Cluster

This function will create a connection pool to your AlloyDB instance using the AlloyDB Python connector. The AlloyDB Python connector will automatically create secure connections to your AlloyDB instance using mTLS.

In [ ]:
import asyncpg

import sqlalchemy
from sqlalchemy.ext.asyncio import AsyncEngine, create_async_engine

from google.cloud.alloydb.connector import AsyncConnector, IPTypes

async def init_connection_pool(connector: AsyncConnector, db_name: str = alloydb_database, pool_size: int = 5) -> AsyncEngine:
    # initialize Connector object for connections to AlloyDB
    connection_string = f"projects/{project_id}/locations/{region}/clusters/{alloydb_cluster}/instances/{alloydb_instance}"

    async def getconn() -> asyncpg.Connection:
        conn: asyncpg.Connection = await connector.connect(
            connection_string,
            "asyncpg",
            user="postgres",
            password=alloydb_password,
            db=db_name,
            ip_type=IPTypes.PRIVATE, # Optionally use IPTypes.PUBLIC
        )
        return conn

    pool = create_async_engine(
        "postgresql+asyncpg://",
        async_creator=getconn,
        pool_size=pool_size,
        max_overflow=0,
        isolation_level='AUTOCOMMIT'
    )
    return pool

connector = AsyncConnector()

postgres_db_pool = await init_connection_pool(connector, "postgres")
finance_db_pool = await init_connection_pool(connector, f"{alloydb_database}")

### Create AlloyDB Database

In [ ]:
# Use postgres_db_pool to create the database
sql = f"CREATE DATABASE {alloydb_database};"
result = await run_alloydb_query(postgres_db_pool, sql)

### Import AlloyDB Data

#### Run the Import

This step generally takes about 5 minutes to complete.

In [ ]:
# Reference: https://cloud.google.com/sdk/gcloud/reference/alloydb/clusters/import
!gcloud alloydb clusters import {alloydb_cluster} --region={region} --database={alloydb_database} --gcs-uri=gs://pr-public-demo-data/adk-toolbox-demo/data/finance.sql --sql --user=postgres

#### Validate Row Counts

In [ ]:
sql = """
SELECT 'users' AS table_name, (SELECT COUNT(*) FROM users) AS imported_count, 2000 AS target_row_count
UNION ALL
SELECT 'mcc_codes', (SELECT COUNT(*) FROM mcc_codes), 109
UNION ALL
SELECT 'transactions', (SELECT COUNT(*) FROM transactions), 13305915
UNION ALL
SELECT 'fraud_labels', (SELECT COUNT(*) FROM fraud_labels), 8914963
UNION ALL
SELECT 'cards', (SELECT COUNT(*) FROM cards), 6146;
"""

await run_alloydb_query(finance_db_pool, sql)

## Configure Least Privilege

### Create a Least Privilege AlloyDB User for MCP Toolbox

MCP Toolbox will use the `toolbox_user` principal to securely access the AlloyDB database. This will be a read-only user to ensure so inadvertent data or object changes are made with our Agent. 

In [ ]:
sql_array = []

sql_array.append(f"""
-- Create the read-only user
CREATE USER toolbox_user WITH PASSWORD '{alloydb_toolbox_user_password}';
""")

sql_array.append("""
-- Grant the user permission to connect to the database
GRANT CONNECT ON DATABASE finance TO toolbox_user;
""")

sql_array.append("""
-- Grant usage on the public schema
GRANT USAGE ON SCHEMA public TO toolbox_user;
""")

sql_array.append("""
-- Grant select (read) access on all existing tables in the public schema
GRANT SELECT ON ALL TABLES IN SCHEMA public TO toolbox_user;
""")

sql_array.append("""
-- Grant usage on all sequences in the public schema
GRANT USAGE ON ALL SEQUENCES IN SCHEMA public TO toolbox_user;
""")

sql_array.append("""
-- Ensure the user gets read access to any new tables created in the future
ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO toolbox_user;
""")

for sql in sql_array:
    await run_alloydb_query(finance_db_pool, sql)

### Spanner Least Privilege for MCP Toolbox

MCP Toolbox will use the credentials of the Cloud Run service account that hosts it to access Spanner. We created a dedicated service account called `toolbox-service-account` via Terraform with the `roles/databaseUser` role for this purpose. 

## Update Dates

Our data set is several years old. We'll update the dates to show off the real-time capabilities of leveraging MCP Toolbox with ADK.

### Update AlloyDB Dates

#### Update Transaction Dates

> NOTE: This update may take 15-20 minutes as there are over 13 Million rows to update in the transactions table.

In [ ]:
# See current MAX and MIN dates
sql = "SELECT MAX(date) AS max_data, MIN(date) AS min_date FROM transactions;"""
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# This query shifts all transaction dates forward. 
# The most recent transaction will be updated to the current time (NOW()),
# and all other transactions will be adjusted proportionally,
# preserving the original time intervals between them. This is done to simulate
# real-time data for the purpose of this lab.
sql = "UPDATE transactions SET date = NOW() - ((SELECT MAX(date) FROM transactions) - date);"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# See new MAX and MIN dates
sql = "SELECT MAX(date) AS max_data, MIN(date) AS min_date FROM transactions;"""
await run_alloydb_query(finance_db_pool, sql)

#### Update Card Dates

In [ ]:
# See current MAX and MIN dates
sql = """
SELECT MAX(acct_open_date) AS max_acct_open_date, 
    MIN(acct_open_date) AS min_acct_open_date,
    MAX(year_pin_last_changed) AS max_year_pin_last_changed, 
    MIN(year_pin_last_changed) AS min_year_pin_last_changed
FROM cards;"""
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# This query shifts all transaction dates forward. 
# The most recent record will be updated to the current time (NOW()),
# and all other record will be adjusted proportionally,
# preserving the original time intervals between them.
sql = """
UPDATE cards
SET
    acct_open_date = (NOW() - ((SELECT MAX(acct_open_date) FROM cards) - acct_open_date) * INTERVAL '1 day')::date,
    year_pin_last_changed = year_pin_last_changed + (EXTRACT(YEAR FROM NOW())::INTEGER - (SELECT MAX(year_pin_last_changed) FROM cards));
"""
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
# See new MAX and MIN dates
sql = """
SELECT MAX(acct_open_date) AS max_acct_open_date, 
    MIN(acct_open_date) AS min_acct_open_date,
    MAX(year_pin_last_changed) AS max_year_pin_last_changed, 
    MIN(year_pin_last_changed) AS min_year_pin_last_changed
FROM cards;"""
await run_alloydb_query(finance_db_pool, sql)

### Update Spanner Dates

#### Verify Spanner Row Counts

> IMPORTANT: If data is missing from Spanner, check the status of the `import-spanner` job in [Dataflow](https://console.cloud.google.com/dataflow/jobs). Resolve any errors and re-run the job before proceeding. 

In [ ]:
sql = """
SELECT 'Account' AS table_name, (SELECT COUNT(*) FROM Account) AS row_count, 700 AS target_row_count
UNION ALL
SELECT 'Loan', (SELECT COUNT(*) FROM Loan), 500
UNION ALL
SELECT 'Person', (SELECT COUNT(*) FROM Person), 500
UNION ALL
SELECT 'AccountTransferAccount', (SELECT COUNT(*) FROM AccountTransferAccount), 500
UNION ALL
SELECT 'AccountRepayLoan', (SELECT COUNT(*) FROM AccountRepayLoan), 500
UNION ALL
SELECT 'PersonOwnAccount', (SELECT COUNT(*) FROM PersonOwnAccount), 700
UNION ALL
SELECT 'AccountAudits', (SELECT COUNT(*) FROM AccountAudits), 500;
"""

result = run_spanner_query(sql)
result

#### View Current Min and Max Dates

In [ ]:
sql = """
SELECT 'Account' AS table_name, MAX(create_time) AS max_time, MIN(create_time) AS min_time FROM Account
UNION ALL
SELECT 'AccountAudits', MAX(audit_timestamp), MIN(audit_timestamp) FROM AccountAudits
UNION ALL
SELECT 'AccountRepayLoan', MAX(create_time), MIN(create_time) FROM AccountRepayLoan
UNION ALL
SELECT 'AccountTransferAccount', MAX(create_time), MIN(create_time) FROM AccountTransferAccount
UNION ALL
SELECT 'Loan', MAX(create_time), MIN(create_time) FROM Loan
UNION ALL
SELECT 'PersonOwnAccount', MAX(create_time), MIN(create_time) FROM PersonOwnAccount;
"""

run_spanner_query(sql)

#### Update Date Columns

In [ ]:
sql_array = []

sql_array.append("""
UPDATE Account
SET create_time = TIMESTAMP_SUB(
  CURRENT_TIMESTAMP(),
  INTERVAL TIMESTAMP_DIFF(
    (SELECT MAX(t.create_time) FROM Account AS t),
    create_time,
    MICROSECOND
  ) MICROSECOND
)
WHERE create_time IS NOT NULL;
""")

sql_array.append("""
UPDATE Loan
SET create_time = TIMESTAMP_SUB(
  CURRENT_TIMESTAMP(),
  INTERVAL TIMESTAMP_DIFF(
    (SELECT MAX(t.create_time) FROM Loan AS t),
    create_time,
    MICROSECOND
  ) MICROSECOND
)
WHERE create_time IS NOT NULL;
""")

sql_array.append("""
UPDATE PersonOwnAccount
SET create_time = TIMESTAMP_SUB(
  CURRENT_TIMESTAMP(),
  INTERVAL TIMESTAMP_DIFF(
    (SELECT MAX(t.create_time) FROM PersonOwnAccount AS t),
    create_time,
    MICROSECOND
  ) MICROSECOND
)
WHERE create_time IS NOT NULL;
""")

for sql in sql_array:
    result = run_spanner_query(sql)
    print(f"{result}\n")

#### Handle Primary Key Column Updates

This code handles a special case where timestampe columns are part of the Primary Key on three of our tables. Spanner does not allow updates on key columns. To work around this, we will:
1. Create temporary tables with the same schema.
2. Copy the data from the original tables to the temporary tables, updating the timestamps in the process.
3. Delete the data from the original tables.
4. Reload the data from the temporary tables back into the original tables.
5. Drop the temporary tables.

#### Create Temp Tables

In [ ]:
ddl_array = []

ddl_array.append("""
CREATE TABLE AccountAuditsTemp (
  id INT64 NOT NULL,
  audit_timestamp TIMESTAMP,
  audit_details STRING(MAX),
) PRIMARY KEY(id, audit_timestamp)
""")

ddl_array.append("""
CREATE TABLE AccountRepayLoanTemp (
  id INT64 NOT NULL,
  loan_id INT64 NOT NULL,
  amount FLOAT64,
  create_time TIMESTAMP NOT NULL,
) PRIMARY KEY(id, loan_id, create_time)
""")

ddl_array.append("""
CREATE TABLE AccountTransferAccountTemp (
  id INT64 NOT NULL,
  to_id INT64 NOT NULL,
  amount FLOAT64,
  create_time TIMESTAMP NOT NULL,
) PRIMARY KEY(id, to_id, create_time)
""")

run_spanner_ddl(ddl_array)

In [ ]:
# Insert data into temp tables with new timestamps
sql_array = [] 

sql_array.append(
"""
INSERT INTO AccountAuditsTemp (id, audit_timestamp, audit_details)
SELECT
    id,
    TIMESTAMP_SUB(
        CURRENT_TIMESTAMP(),
        INTERVAL TIMESTAMP_DIFF(
            (SELECT MAX(t.audit_timestamp) FROM AccountAudits AS t),
            audit_timestamp,
            MICROSECOND
        ) MICROSECOND
    ) AS audit_timestamp,
    audit_details
FROM
    AccountAudits
"""
)

sql_array.append("""
INSERT INTO AccountRepayLoanTemp (id, loan_id, amount, create_time)
SELECT
    id,
    loan_id,
    amount,
    TIMESTAMP_SUB(
        CURRENT_TIMESTAMP(),
        INTERVAL TIMESTAMP_DIFF(
            (SELECT MAX(t.create_time) FROM AccountRepayLoan AS t),
            create_time,
            MICROSECOND
        ) MICROSECOND
    ) AS create_time
FROM
    AccountRepayLoan
""")

sql_array.append("""
INSERT INTO AccountTransferAccountTemp (id, to_id, amount, create_time)
SELECT
    id,
    to_id,
    amount,
    TIMESTAMP_SUB(
        CURRENT_TIMESTAMP(),
        INTERVAL TIMESTAMP_DIFF(
            (SELECT MAX(t.create_time) FROM AccountTransferAccount AS t),
            create_time,
            MICROSECOND
        ) MICROSECOND
    ) AS create_time
FROM
    AccountTransferAccount
""")

for sql in sql_array:
    run_spanner_query(sql)

In [ ]:
# View timestamp data in temp tables

sql = """
SELECT 'AccountAuditsTemp' AS table_name, MAX(audit_timestamp) AS max_time, MIN(audit_timestamp) AS min_time FROM AccountAuditsTemp
UNION ALL
SELECT 'AccountRepayLoanTemp', MAX(create_time), MIN(create_time) FROM AccountRepayLoanTemp
UNION ALL
SELECT 'AccountTransferAccountTemp', MAX(create_time), MIN(create_time) FROM AccountTransferAccountTemp
"""
run_spanner_query(sql)

#### Reload Data into Source Tables

In [ ]:
# Delete data in source tables
sql_array = []

sql_array.append("DELETE FROM AccountAudits WHERE 1 = 1")

sql_array.append("DELETE FROM AccountRepayLoan WHERE 1 = 1")

sql_array.append("DELETE FROM AccountTransferAccount WHERE 1 = 1")

for sql in sql_array:
    run_spanner_query(sql)

In [ ]:
# Load source tables from temp tables

sql_array = []

sql_array.append(
"""
INSERT INTO AccountAudits (id, audit_timestamp, audit_details)
SELECT
    id,
    audit_timestamp,
    audit_details
FROM
    AccountAuditsTemp
"""
)

sql_array.append("""
INSERT INTO AccountRepayLoan (id, loan_id, amount, create_time)
SELECT
    id,
    loan_id,
    amount,
    create_time
FROM
    AccountRepayLoanTemp
""")

sql_array.append("""
INSERT INTO AccountTransferAccount (id, to_id, amount, create_time)
SELECT
    id,
    to_id,
    amount,
    create_time
FROM
    AccountTransferAccountTemp
""")

for sql in sql_array:
    run_spanner_query(sql)

#### View New Min and Max Dates

In [ ]:
sql = """
SELECT 'Account' AS table_name, MAX(create_time) AS max_time, MIN(create_time) AS min_time FROM Account
UNION ALL
SELECT 'AccountAudits', MAX(audit_timestamp), MIN(audit_timestamp) FROM AccountAudits
UNION ALL
SELECT 'AccountRepayLoan', MAX(create_time), MIN(create_time) FROM AccountRepayLoan
UNION ALL
SELECT 'AccountTransferAccount', MAX(create_time), MIN(create_time) FROM AccountTransferAccount
UNION ALL
SELECT 'Loan', MAX(create_time), MIN(create_time) FROM Loan
UNION ALL
SELECT 'PersonOwnAccount', MAX(create_time), MIN(create_time) FROM PersonOwnAccount;
"""

run_spanner_query(sql)

#### Clean Up Temp Tables

In [ ]:
ddl_array = []

ddl_array.append("""
DROP TABLE AccountAuditsTemp
""")

ddl_array.append("""
DROP TABLE AccountRepayLoanTemp
""")

ddl_array.append("""
DROP TABLE AccountTransferAccountTemp
""")

run_spanner_ddl(ddl_array)

#### Verify Spanner Row Counts

In [ ]:
sql = """
SELECT 'Account' AS table_name, (SELECT COUNT(*) FROM Account) AS row_count, 700 AS target_row_count
UNION ALL
SELECT 'Loan', (SELECT COUNT(*) FROM Loan), 500
UNION ALL
SELECT 'Person', (SELECT COUNT(*) FROM Person), 500
UNION ALL
SELECT 'AccountTransferAccount', (SELECT COUNT(*) FROM AccountTransferAccount), 500
UNION ALL
SELECT 'AccountRepayLoan', (SELECT COUNT(*) FROM AccountRepayLoan), 500
UNION ALL
SELECT 'PersonOwnAccount', (SELECT COUNT(*) FROM PersonOwnAccount), 700
UNION ALL
SELECT 'AccountAudits', (SELECT COUNT(*) FROM AccountAudits), 500;
"""

result = run_spanner_query(sql)
result

## Explore AlloyDB Data

### Review AlloyDB Schema

The AlloyDB database contains data related to transactions, credit cards, users, mcc codes, and historical fraud cases. See the ERD diagram below for details on the tables, columns, and relationships. 

![AlloyDB Schema](img/alloydb_finance_db_erd.png)

### Run AlloyDB Queries

In [ ]:
sql = "SELECT * FROM transactions LIMIT 10;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
sql = "SELECT * FROM cards LIMIT 10;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
sql = "SELECT * FROM users LIMIT 10;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
sql = "SELECT * FROM mcc_codes LIMIT 10;"
await run_alloydb_query(finance_db_pool, sql)

In [ ]:
sql = "SELECT * FROM fraud_labels LIMIT 10;"
await run_alloydb_query(finance_db_pool, sql)

## Explore Spanner Data

### Review the Spanner Schema

The Spanner dataset is already loaded for you via Terraform automation. It contains data about accounts, account transfers, account audits, loans, and loan repayments.

The schema is defined as follows:

```
CREATE TABLE Account (
  id INT64 NOT NULL,
  create_time TIMESTAMP,
  is_blocked BOOL,
  type STRING(MAX),
) PRIMARY KEY(id);

CREATE TABLE AccountAudits (
  id INT64 NOT NULL,
  audit_timestamp TIMESTAMP,
  audit_details STRING(MAX),
) PRIMARY KEY(id, audit_timestamp),
  INTERLEAVE IN PARENT Account ON DELETE CASCADE;

CREATE TABLE AccountRepayLoan (
  id INT64 NOT NULL,
  loan_id INT64 NOT NULL,
  amount FLOAT64,
  create_time TIMESTAMP NOT NULL,
) PRIMARY KEY(id, loan_id, create_time),
  INTERLEAVE IN PARENT Account ON DELETE CASCADE;

CREATE TABLE AccountTransferAccount (
  id INT64 NOT NULL,
  to_id INT64 NOT NULL,
  amount FLOAT64,
  create_time TIMESTAMP NOT NULL,
) PRIMARY KEY(id, to_id, create_time),
  INTERLEAVE IN PARENT Account ON DELETE CASCADE;

CREATE TABLE Loan (
  id INT64 NOT NULL,
  loan_amount FLOAT64,
  balance FLOAT64,
  create_time TIMESTAMP,
  interest_rate FLOAT64,
) PRIMARY KEY(id);

CREATE TABLE Person (
  id INT64 NOT NULL,
  name STRING(MAX),
) PRIMARY KEY(id);

CREATE TABLE PersonOwnAccount (
  id INT64 NOT NULL,
  account_id INT64 NOT NULL,
  create_time TIMESTAMP,
) PRIMARY KEY(id, account_id),
  INTERLEAVE IN PARENT Person ON DELETE CASCADE;

CREATE PROPERTY GRAPH FinGraph
  NODE TABLES(
    Account
      KEY(id)
      LABEL Account PROPERTIES(
        create_time,
        id,
        is_blocked,
        type),

    Loan
      KEY(id)
      LABEL Loan PROPERTIES(
        balance,
        create_time,
        id,
        interest_rate,
        loan_amount),

    Person
      KEY(id)
      LABEL Person PROPERTIES(
        id,
        name)
  )
  EDGE TABLES(
    AccountRepayLoan
      KEY(id, loan_id, create_time)
      SOURCE KEY(id) REFERENCES Account(id)
      DESTINATION KEY(loan_id) REFERENCES Loan(id)
      LABEL Repays PROPERTIES(
        amount,
        create_time,
        id,
        loan_id),

    AccountTransferAccount
      KEY(id, to_id, create_time)
      SOURCE KEY(id) REFERENCES Account(id)
      DESTINATION KEY(to_id) REFERENCES Account(id)
      LABEL Transfers PROPERTIES(
        amount,
        create_time,
        id,
        to_id),

    PersonOwnAccount
      KEY(id, account_id)
      SOURCE KEY(id) REFERENCES Person(id)
      DESTINATION KEY(account_id) REFERENCES Account(id)
      LABEL Owns PROPERTIES(
        account_id,
        create_time,
        id)
  );
```

Notice that the Spanner schema takes advantage of both the core relational database model, as well as Spanner's built-in property graph model. The high-level graph nodes and edges are visualized below. 

![Spanner Graph Schema](img/spanner_graph_schema.png)

### Run Relational Spanner Queries

In [ ]:
sql = "SELECT * FROM Account LIMIT 10;"
run_spanner_query(sql)

In [ ]:
sql = "SELECT * FROM AccountAudits LIMIT 10;"
run_spanner_query(sql)

In [ ]:
sql = "SELECT * FROM AccountRepayLoan LIMIT 10;"
run_spanner_query(sql)

In [ ]:
sql = "SELECT * FROM AccountTransferAccount LIMIT 10;"
run_spanner_query(sql)

In [ ]:
sql = "SELECT * FROM Loan LIMIT 10;"
run_spanner_query(sql)

In [ ]:
sql = "SELECT * FROM Person LIMIT 10;"
run_spanner_query(sql)

In [ ]:
sql = "SELECT * FROM Account LIMIT 10;"
run_spanner_query(sql)

### Run Spanner Graph Queries

Read more about running graph queries on Spanner in the [Spanner docs](https://cloud.google.com/spanner/docs/graph/queries-overview).

In [ ]:
sql = """
GRAPH FinGraph
MATCH
  (person:Person {name: "Jacoby"})-[own:Owns]->
  (account:Account)-[repay:Repays]->(loan:Loan)
RETURN
  account.id AS account_id,
  repay.create_time AS repay_time,
  repay.amount AS loan_repay_amount,
  loan.id AS loan_id,
  loan.loan_amount AS loan_amount
ORDER BY repay.create_time;
"""
run_spanner_query(sql)

In [ ]:
sql = """
GRAPH FinGraph
MATCH ANY SHORTEST
  (src_accnt:Account {id:75} )-[transfers:Transfers]->{3,6}
  (dst_accnt:Account {id:199})
RETURN
  ARRAY_LENGTH(transfers) AS num_hops,
  TO_JSON(transfers) AS transfer_edges;
"""

run_spanner_query(sql)

Congratulations, you have completed Module 1! Proceed to [`2_deploy_mcp_toolbox.ipynb`](./2_deploy_mcp_toolbox.ipynb) to configure and deploy MCP Toolbox.